In [2]:
from opt.mc_sim import *
from common.variables import *
from sim.sim_functions import *
import torch
import gym
from gym import spaces
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete, Tuple, MultiBinary

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
s = SourcingEnv()

In [4]:
class CustomGymEnv(Env):
    #metadata = {'render.modes': ['human']}  
    
    def __init__(self, sourcing_env):
        self.SourcingEnv = sourcing_env
        self.counter = 0
        
        # Actions we can take, down, stay, up
        self.action_space = MultiDiscrete([30,30])

        # Inventory Observation State
        self.observation_space = Box(low=np.array([-30, 0, 0, 0, 0]), high=np.array([30, 30, 30, 1, 1]), shape=(5,), dtype=int)
        #Tuple(Box(-30,30,shape=(1,), dtype=int), Discrete(30), Discrete(30), MultiBinary(2))
                                       
    
    def step(self, action):        
        reward = self.reward_func(self.SourcingEnv.current_state, action)
        next_state, event, i, event_probs, supplier_index = self.SourcingEnv.step(action)
        self.counter += 1
        
        info = {}
        
        if self.counter < PERIODS:
            done = False
        else:
            done = True
        
        next_state_array = np.array(next_state.get_list_repr())
        return next_state_array, reward, done, info
    
    def reset(self):
        self.SourcingEnv = SourcingEnv()
        return np.array(self.SourcingEnv.current_state.get_list_repr())
        
    def reward_func(self, state, action):
        reward_hb = H_COST * state.s if state.s >= 0 else  -B_PENALTY * state.s 
        reward = reward_hb + np.sum(np.multiply(action, PROCUREMENT_COST_VEC))
        reward = float(reward)
        return -reward
    
    #def render(self):
        # Implement viz
       # pass

In [5]:
Discrete(INVEN_LIMIT)

Discrete(30)

In [6]:
MultiDiscrete([5,5]).sample()

array([4, 2])

In [7]:
s = SourcingEnv()

In [8]:
custom_gym_env = CustomGymEnv(s)

In [9]:
m_state = custom_gym_env.SourcingEnv.current_state
m_state.get_list_repr()

[0, 0, 0, 1, 1]

In [10]:
str(m_state)

'Stock: 0, n backorders: [0. 0.], supplier status (on/off): [1. 1.]'

In [11]:
PERIODS

10

In [12]:
custom_gym_env.step(np.array([1,3]))

(array([0, 1, 3, 1, 1]), -60.0, False, {})

In [13]:
from stable_baselines3.common.env_checker import check_env

In [14]:
check_env(custom_gym_env) #warn=True)

In [15]:
episodes = 15
for episode in range(1, episodes+1):
    state = custom_gym_env.reset()
    done = False
    cost = 0
    
    
    while not done:
        #env.render()
        action = custom_gym_env.action_space.sample()
        n_state, reward, done, info = custom_gym_env.step(action)
        cost+=reward
        observation = custom_gym_env.step(action)
    print('Episode:{} Cost:{} Observation {}' .format(episode, cost, observation))
    

custom_gym_env.close()

Episode:1 Cost:-300.0 Observation (array([15, 10, 15,  1,  1]), -300.0, True, {})
Episode:2 Cost:-765.0 Observation (array([18, 30, 18,  1,  1]), -765.0, True, {})
Episode:3 Cost:-520.0 Observation (array([14, 20, 14,  1,  1]), -520.0, True, {})
Episode:4 Cost:-1140.0 Observation (array([23, 23, 42,  1,  1]), -1140.0, True, {})
Episode:5 Cost:-385.0 Observation (array([ 8,  8, 10,  1,  1]), -385.0, True, {})
Episode:6 Cost:-670.0 Observation (array([14, 14, 16,  1,  1]), -670.0, True, {})
Episode:7 Cost:-980.0 Observation (array([19, 19, 50,  1,  1]), -980.0, True, {})
Episode:8 Cost:-955.0 Observation (array([19, 19, 40,  1,  1]), -955.0, True, {})
Episode:9 Cost:-1315.0 Observation (array([27, 27, 40,  1,  1]), -1315.0, True, {})
Episode:10 Cost:-465.0 Observation (array([21, 16, 21,  1,  1]), -465.0, True, {})
Episode:11 Cost:-515.0 Observation (array([10, 10, 26,  1,  1]), -515.0, True, {})
Episode:12 Cost:-1240.0 Observation (array([25, 25, 46,  1,  1]), -1240.0, True, {})
Episode

In [16]:
custom_gym_env.observation_space

Box([-30   0   0   0   0], [30 30 30  1  1], (5,), int64)

In [17]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [18]:
log_path = os.path.join('Training', 'Logs')

In [19]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [20]:
model.learn(total_timesteps=100)

Logging to Training/Logs/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -673     |
| time/              |          |
|    fps             | 1174     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [21]:
model.learn(total_timesteps=100)

Logging to Training/Logs/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -673     |
| time/              |          |
|    fps             | 1178     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [22]:
model.learn(total_timesteps=100)

Logging to Training/Logs/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -570     |
| time/              |          |
|    fps             | 1153     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [23]:
model.learn(total_timesteps=100)

Logging to Training/Logs/PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -456     |
| time/              |          |
|    fps             | 1171     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [24]:
model.learn(total_timesteps=100)

Logging to Training/Logs/PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -442     |
| time/              |          |
|    fps             | 1173     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [25]:
model.learn(total_timesteps=100)

Logging to Training/Logs/PPO_6
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -425     |
| time/              |          |
|    fps             | 1193     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [26]:
model.learn(total_timesteps=100)

Logging to Training/Logs/PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -257     |
| time/              |          |
|    fps             | 1191     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [27]:
model.learn(total_timesteps=100)

Logging to Training/Logs/PPO_8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -244     |
| time/              |          |
|    fps             | 1177     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [28]:
model.learn(total_timesteps=100)

Logging to Training/Logs/PPO_9
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -226     |
| time/              |          |
|    fps             | 1212     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [29]:
model.learn(total_timesteps=100)

Logging to Training/Logs/PPO_10
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -183     |
| time/              |          |
|    fps             | 1184     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [33]:
custom_gym_env.step(action)

(array([ 0, 18, 25,  1,  1]), -935.0, True, {})

In [30]:
model.save('PPO')

In [32]:
evaluate_policy(model, custom_gym_env, n_eval_episodes=10, render=False)

(0.0, 0.0)

In [ ]:
import matplotlib.pyplot as plt